# NYC Bike Sharing Linear Regression

The following steps will show you how to train, evaluate and test a linear regression model using BigQuery ML.

## Data Preparation

In [2]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

dataset_id = "{}.12_notebook".format(client.project)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# Geographic location where the dataset should reside.
dataset.location = "US"

# Dataset creation
dataset = client.create_dataset(dataset, timeout=30)  
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset bigqueryml-packt.12_notebook


In [3]:
%%bigquery

### Creation of the training table ###
CREATE OR REPLACE TABLE `12_notebook.training_table` AS
              SELECT 
                    tripduration/60 tripduration,
					starttime,
					stoptime,
					start_station_id,
					start_station_name,
					start_station_latitude,
					start_station_longitude,
					end_station_id,
					end_station_name,
					end_station_latitude,
					end_station_longitude,
					bikeid,
					usertype,
					birth_year,
					gender,
					customer_plan
              FROM
                    `bigquery-public-data.new_york_citibike.citibike_trips`
              WHERE 
                    (
                        (EXTRACT (YEAR FROM starttime)=2017 AND
                          (EXTRACT (MONTH FROM starttime)>=04 OR EXTRACT (MONTH FROM starttime)<=10))
                        OR (EXTRACT (YEAR FROM starttime)=2018 AND
                          (EXTRACT (MONTH FROM starttime)>=01 OR EXTRACT (MONTH FROM starttime)<=02))
                    )
                    AND (tripduration>=3*60 AND tripduration<=3*60*60)
                    AND  birth_year is not NULL
                    AND birth_year < 2007;

""


In [4]:
%%bigquery

### Creation of the evaluation table ###
CREATE OR REPLACE TABLE  `12_notebook.evaluation_table` AS
SELECT 
                    tripduration/60 tripduration,
					starttime,
					stoptime,
					start_station_id,
					start_station_name,
					start_station_latitude,
					start_station_longitude,
					end_station_id,
					end_station_name,
					end_station_latitude,
					end_station_longitude,
					bikeid,
					usertype,
					birth_year,
					gender,
					customer_plan
				FROM
                    `bigquery-public-data.new_york_citibike.citibike_trips`
				WHERE 
                    (EXTRACT (YEAR FROM starttime)=2018 AND (EXTRACT (MONTH FROM starttime)=03 OR EXTRACT (MONTH FROM starttime)=04))
                    AND (tripduration>=3*60 AND tripduration<=3*60*60)
                    AND  birth_year is not NULL
                    AND birth_year < 2007;

""


In [5]:
%%bigquery

### Creation of the prediction table ###
CREATE OR REPLACE TABLE  `12_notebook.prediction_table` AS
              SELECT 
                   tripduration/60 tripduration,
					starttime,
					stoptime,
					start_station_id,
					start_station_name,
					start_station_latitude,
					start_station_longitude,
					end_station_id,
					end_station_name,
					end_station_latitude,
					end_station_longitude,
					bikeid,
					usertype,
					birth_year,
					gender,
					customer_plan
				FROM
                    `bigquery-public-data.new_york_citibike.citibike_trips`
              WHERE 
                    EXTRACT (YEAR FROM starttime)=2018
                    AND EXTRACT (MONTH FROM starttime)=05
                     AND (tripduration>=3*60 AND tripduration<=3*60*60)
                    AND  birth_year is not NULL
                    AND birth_year < 2007;

""


## Training the linear regression

In [6]:
%%bigquery

CREATE OR REPLACE MODEL `12_notebook.linear_regression_notebook`
OPTIONS
  (model_type='linear_reg') AS
SELECT
  start_station_name,
  end_station_name,
   IF (EXTRACT(DAYOFWEEK FROM starttime)=1 OR EXTRACT(DAYOFWEEK FROM starttime)=7, true, false) is_weekend,
  tripduration as label
FROM
  `12_notebook.training_table`;

""


## Evaluating the linear regression

In [7]:
%%bigquery

SELECT
  *
FROM
  ML.EVALUATE(MODEL `12_notebook.linear_regression_notebook`,
    (
    SELECT
          start_station_name,
          end_station_name,
          IF (EXTRACT(DAYOFWEEK FROM starttime)=1 OR EXTRACT(DAYOFWEEK FROM starttime)=7, true, false) is_weekend,
           tripduration as label
    FROM
           `12_notebook.evaluation_table`));

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,7.124097,119.461055,0.377953,5.51093,0.095371,0.095479


## Testing the linear regression

In [8]:
%%bigquery

SELECT
   tripduration as actual_duration,
   predicted_label as predicted_duration,
   ABS(tripduration - predicted_label) difference_in_min
FROM
  ML.PREDICT(MODEL `12_notebook.linear_regression_notebook`,
    (
    SELECT
          start_station_name,
          end_station_name,
          IF (EXTRACT(DAYOFWEEK FROM starttime)=1 OR EXTRACT(DAYOFWEEK FROM starttime)=7, true, false) is_weekend,
           tripduration
    FROM
           `12_notebook.prediction_table`
    ))
    order by  difference_in_min asc;

,actual_duration,predicted_duration,difference_in_min
0,16.616667,16.616668,9.775469e-07
1,15.683333,15.683348,1.502998e-05
2,12.233333,12.233349,1.526299e-05
3,12.133333,12.133318,1.571753e-05
4,12.800000,12.800024,2.423694e-05
...,...,...,...
1728073,179.316667,11.867817,1.674489e+02
1728074,179.350000,11.151197,1.681988e+02
1728075,178.316667,9.539430,1.687772e+02
1728076,178.916667,9.539430,1.693772e+02
